# Counting qubits: Zephyr vs Pegasus

Notebook that maps a Hamiltonian defined through truncated and penalization embeddings on both the Pegasus and the Zephyr topology to comparate the number of needed physical qubits.

Reference: [Solving systems of Boolean multivariate equations with quantum annealing](https://arxiv.org/abs/2111.13224) by Sergi Ramos-Calderer, Carlos Bravo-Prieto, Ruge Lin, Emanuele Bellini, Marc Manzano, Najwa Aaraj, and José I. Latorre.

## Setup

In [26]:
# import
import my_lib as lib 
import article_lib as article

import examples_anf as anf
import examples_nnf as nnf

import numpy as np
import pandas as pd 
from sympy import symbols
from dwave.inspector import show

In [27]:
## bit list

bit_max = 10                                        # max number of bits to run
bit_list = [i for i in range(4, bit_max+1, 2)]      # even numbers from 4 to bit_max

p_list = []
x_list = []

for bits in bit_list:

    x = symbols(" ".join((f"x{i}" for i in range(1, bits+1))))
    
    if bits == 4:
        p, sol = anf.example_4_anf(*x)
    elif bits == 6:
        p, sol = anf.example_6_anf(*x)
    elif bits == 8:
        p, sol = anf.example_8_anf(*x)
    elif bits == 10:
        p, sol = anf.example_10_anf(*x)
    elif bits == 12:
        p, sol = anf.example_12_anf(*x)

    p_list.append(p)
    x_list.append(x)

## Truncated Embedding

### Building the Hamiltonian

Since we will run the truncated Hamiltonians both via Pegasus and Zephyr topology, it's best to define them a priori and store them away.

They will be added to a designated list already in QUBO format whilist their offsets will be stored in another list, in the same order.

In [3]:
## building truncated H

trunc_Hs = []
trunc_offsets = []

for i in range(len(bit_list)):

    # symbolic def
    truncated = article.truncated_embedding(bit_list[i], p_list[i], x_list[i])
    trunc_H = truncated.create_hamiltonian()
    trunc_sym = truncated.get_symbols()

    # QUBO model
    d_art = article.dwave_annealing(trunc_H, bit_list[i], trunc_sym)
    trunc_H_qubo, trunc_qubo_offset = d_art.symbolic_to_dwave(trunc_H, d_art.get_symbol_num(trunc_sym))

    # store
    trunc_Hs.append(trunc_H_qubo)
    trunc_offsets.append(trunc_qubo_offset)

### Pegasus

In [4]:
p_trunc_qubits = {}

for i in range(len(bit_list)):

    trunc_runner = lib.dwave_runners(trunc_Hs[i], trunc_offsets[i], 
                                     bit_list[i], topology='Pegasus', 
                                     chosen_chainstrength='Article')
    
    logical_qubits, physical_qubits = trunc_runner.counting_qubits(average=4)

    p_trunc_qubits[f'{bit_list[i]}_bits'] = [logical_qubits, physical_qubits]

Building the BQM model.
Running on Pegasus Topology.
You chose the Article chainstrength: 27.530612244897956.
Finished running the experiment #1!
Finished running the experiment #2!
Finished running the experiment #3!
Finished running the experiment #4!

--------------------------------------------------------------------------

Building the BQM model.
Running on Pegasus Topology.
You chose the Article chainstrength: 28.797385620915033.
Finished running the experiment #1!
Finished running the experiment #2!
Finished running the experiment #3!
Finished running the experiment #4!

--------------------------------------------------------------------------

Building the BQM model.
Running on Pegasus Topology.
You chose the Article chainstrength: 30.021599999999996.
Finished running the experiment #1!
Finished running the experiment #2!
Finished running the experiment #3!
Finished running the experiment #4!

--------------------------------------------------------------------------

Buildin

In [5]:
p_trunc_qubits_df = pd.DataFrame(p_trunc_qubits, index=['logical_qubits', 'physical_qubits'])
print('\033[1m' + '-----------Used Qubits on Pegasus-----------' + '\033[0m')
print(p_trunc_qubits_df)

-----------Used Qubits on Pegasus-----------
                 4_bits  6_bits  8_bits  10_bits
logical_qubits     30.0    90.0   231.0    451.0
physical_qubits    56.5   222.5   733.5   1661.0


### Zephyr

In [7]:
z_trunc_qubits = {}

for i in range(len(bit_list)):

    trunc_runner = lib.dwave_runners(trunc_Hs[i], trunc_offsets[i], 
                                     bit_list[i], topology='Zephyr', 
                                     chosen_chainstrength='Article')
    
    logical_qubits, physical_qubits = trunc_runner.counting_qubits(average=4)

    z_trunc_qubits[f'{bit_list[i]}_bits'] = [logical_qubits, physical_qubits]
    max_i = i

Building the BQM model.
Running on Zephyr Topology.
You chose the Article chainstrength: 27.530612244897956.
Finished running the experiment #1!
Finished running the experiment #2!
Finished running the experiment #3!
Finished running the experiment #4!

--------------------------------------------------------------------------

Building the BQM model.
Running on Zephyr Topology.
You chose the Article chainstrength: 28.797385620915033.
Finished running the experiment #1!
Finished running the experiment #2!
Finished running the experiment #3!
Finished running the experiment #4!

--------------------------------------------------------------------------

Building the BQM model.
Running on Zephyr Topology.
You chose the Article chainstrength: 30.021599999999996.
Finished running the experiment #1!
Finished running the experiment #2!
Finished running the experiment #3!
Finished running the experiment #4!

--------------------------------------------------------------------------

Building t

ValueError: no embedding found

Due to hardware restriction [1], we can only compute the number of physical qubits up to a problem with $10$ original variables. 

We will manually add this information in order to create the Pandas dataframe. Notice that we can reuse the number of logical qubits from the Pegasus runs, considering that this number only depends on the formulation of the Hamiltonian (which is independent from the topology).

[1] Currently available Zephyr prototype has roughly $1200$ qubits, against the first prototype that only had $\sim 500$ qubits.

In [8]:
## completing info

for j in range(max_i+1, len(bit_list)):
    z_trunc_qubits[f'{bit_list[i]}_bits'] = [p_trunc_qubits[f'{bit_list[i]}_bits'][0], None]

In [9]:
z_trunc_qubits_df = pd.DataFrame(z_trunc_qubits, index=['logical_qubits', 'physical_qubits'])
print('\033[1m' + '-----------Used Qubits on Zephyr-----------' + '\033[0m')
print(z_trunc_qubits_df)

-----------Used Qubits on Zephyr-----------
                 4_bits  6_bits  8_bits  10_bits
logical_qubits    30.00    90.0  231.00    451.0
physical_qubits   50.25   211.0  667.75      NaN


## Penalization Embedding

### Building the Hamiltonian

In [ ]:
## building penalization H

pen_Hs = []
pen_offsets = []

for i in range(len(bit_list)):

    # symbolic def
    penalization = article.penalization_embedding(bit_list[i], p_list[i], x_list[i])
    pen_H = penalization.create_hamiltonian()
    pen_sym = penalization.get_symbols()
    print(pen_sym)

    # QUBO model
    d_art = article.dwave_annealing(pen_H, bit_list[i], pen_sym)
    pen_H_qubo, pen_qubo_offset = d_art.symbolic_to_dwave(pen_H, d_art.get_symbol_num(pen_sym))

    # store
    pen_Hs.append(pen_H_qubo)
    pen_offsets.append(pen_qubo_offset)

### Pegasus

In [15]:
p_pen_qubits = {}

for i in range(len(bit_list)):

    pen_runner = lib.dwave_runners(pen_Hs[i], pen_offsets[i], 
                                    bit_list[i], topology='Pegasus', 
                                    chosen_chainstrength='Article')
    
    logical_qubits, physical_qubits = pen_runner.counting_qubits(average=4)

    p_pen_qubits[f'{bit_list[i]}_bits'] = [logical_qubits, physical_qubits]

Building the BQM model.
Running on Pegasus Topology.
You chose the Article chainstrength: 28.935483870967744.
Finished running the experiment #1!
Finished running the experiment #2!
Finished running the experiment #3!
Finished running the experiment #4!

--------------------------------------------------------------------------

Building the BQM model.
Running on Pegasus Topology.
You chose the Article chainstrength: 29.934782608695652.
Finished running the experiment #1!
Finished running the experiment #2!
Finished running the experiment #3!
Finished running the experiment #4!

--------------------------------------------------------------------------

Building the BQM model.
Running on Pegasus Topology.
You chose the Article chainstrength: 31.689342403628117.
Finished running the experiment #1!
Finished running the experiment #2!
Finished running the experiment #3!
Finished running the experiment #4!

--------------------------------------------------------------------------

Buildin

In [16]:
p_pen_qubits_df = pd.DataFrame(p_pen_qubits, index=['logical_qubits', 'physical_qubits'])
print('\033[1m' + '-----------Used Qubits on Pegasus-----------' + '\033[0m')
print(p_pen_qubits_df)

-----------Used Qubits on Pegasus-----------
                 4_bits  6_bits  8_bits  10_bits
logical_qubits     61.0  150.00  345.00   645.00
physical_qubits   105.0  310.25  891.25  1805.25


### Zephyr

In [18]:
z_pen_qubits = {}

for i in range(len(bit_list)):

    pen_runner = lib.dwave_runners(pen_Hs[i], pen_offsets[i], 
                                    bit_list[i], topology='Zephyr', 
                                    chosen_chainstrength='Article')
    
    logical_qubits, physical_qubits = pen_runner.counting_qubits(average=4)

    z_pen_qubits[f'{bit_list[i]}_bits'] = [logical_qubits, physical_qubits]
    max_i = i

Building the BQM model.
Running on Zephyr Topology.
You chose the Article chainstrength: 28.935483870967744.
Finished running the experiment #1!
Finished running the experiment #2!
Finished running the experiment #3!
Finished running the experiment #4!

--------------------------------------------------------------------------

Building the BQM model.
Running on Zephyr Topology.
You chose the Article chainstrength: 29.934782608695652.
Finished running the experiment #1!
Finished running the experiment #2!
Finished running the experiment #3!
Finished running the experiment #4!

--------------------------------------------------------------------------

Building the BQM model.
Running on Zephyr Topology.
You chose the Article chainstrength: 31.689342403628117.
Finished running the experiment #1!
Finished running the experiment #2!
Finished running the experiment #3!
Finished running the experiment #4!

--------------------------------------------------------------------------

Building t

ValueError: no embedding found

In [19]:
## completing info

for j in range(max_i+1, len(bit_list)):
    z_pen_qubits[f'{bit_list[i]}_bits'] = [p_pen_qubits[f'{bit_list[i]}_bits'][0], None]

In [20]:
z_pen_qubits_df = pd.DataFrame(z_pen_qubits, index=['logical_qubits', 'physical_qubits'])
print('\033[1m' + '-----------Used Qubits on Zephyr-----------' + '\033[0m')
print(z_pen_qubits_df)

-----------Used Qubits on Zephyr-----------
                 4_bits  6_bits  8_bits  10_bits
logical_qubits     61.0   150.0   345.0    645.0
physical_qubits    88.5   279.5   769.0      NaN
